# Deciphering the generated Fact table:


*  This includes concept sets that are automatically added to all patient datasets.

These concept sets were created by other team members and are based on clinical diagnoses that generally align with overall diagnostic criteria.


The objective of this project is to isolate and analyze records pertaining exclusively to delivery patients.

In [ ]:
# Purpose: The purpose of this table is to create transform of the all_patients_fact_day_table_de_id_ We are going to add patients that delivered and its 1 and pregnancy is 1. This is to narrow the cohort.
# Creator: for the Team1 project
# Created/Updated 04/01/2025

# Description: Used delivery records only.
# - This code does not include pregnancies that involved abortions or complications. The cohorts are strictly categorized with a value of 1 for both delivery mode and pregnancy.

# - The Value 0 means Missing data or any event that happend and person didn't deliver. Adding the additional sorting will help us to include only FEMALE and exclude MALE. This can be done at the colpsed table at the end.
# - ! The missing data is represented as 0 so filtering dawn on pregnancy as 1 where the pregnancy was recorded at the visit. We will then exclud missing data from the cohort.
# - It filters the DataFrame to include only those rows where the column delivery_record_only equals 1. This step isolates patients who have a delivery record.
# - Pulling all delivery patients form the data makse more sens adding the information abot number of givenbirths at this moment
# combin all filtering on one node that is ithe seaiset wayt o avoid the program bugs, if you want ot filter in the next node you need to recall the first node as dataframe df




from pyspark.sql import functions as F

def Delivery_index_date_setup_1(all_patients_fact_day_table_de_id):
    df = all_patients_fact_day_table_de_id

    # Print the original sample size
    print("Original sample size:", df.count())

    # Filter for rows where delivery_record_only == 1 or number_given_birth == 1
    filtered_df = df.filter(
    (F.col("delivery_record_only") == 1) |
    (F.col("number_given_birth") == 1) |
    (F.col("pregnancy") ==1)
    )

    # Further filter where pregnancy == 1
    # filtered_df = filtered_df.filter(F.col("pregnancy") == 1)
    # Print the filtered sample size
    print("Filtered sample size:", filtered_df.count())
    return filtered_df





This Follwong code will allow us to filter data further as wee see that pragnancy conceptset gives us 1 or 0. which means 0 there might not be record or missing data that NA is given as 0. Additional filterinif done separetly will not take into account the previous filtering.

---



In [ ]:
# Purpose: The purpose of this table is to create transform of the all_patients_fact_day_table_de_id_ We are going to add patients that delivered and its 1 and pregnancy is 1. This is to narrow the cohort.
# Creator: for the Team1 project
# Created/Updated 04/01/2025
# Desctiprion: Pregnancy as 1 set for our incusion criteria. This code is implementing pregnancy as 1.

from pyspark.sql import functions as F

def Only_Pregnant_as1(Delivery_index_date_setup_1):
    df = Delivery_index_date_setup_1

    # Print the original sample siz
    print("Original1 sample size:", df.count())

    # Filter for rows where delivery_record_only == 1
    filtered_df = df.filter (F.col("pregnancy") == 1)

    # Print the filtered sample size
    print("Filtered2 sample size:", filtered_df.count())


    return filtered_df


## Setting up delivery date . Where we are intersted in the first delivery date, simple and less heavy no addition of extra columns

In [ ]:
# Purpose: The purpose of this table is to create transform of the Delivery to Only_Pregnant_as1_
# Creator: for the Team1 project
# Created/Updated 04/03/2025
# Description: As the DElivery Table ony contains the Delivery as 1 and pregnancy as 1. The code dosn't take the other virables to narrow donw to pregnancy. I have created a separate node to narrow down the delivery_1 to Pregnancy_1. I think we should sort first as a  pregnancy and from that cohort narrow down the deliery. However, in pregnancy table 0 means missing data for the explicite date of the visit for the paptient.




from pyspark.sql import functions as F


def DDateSetup( Only_Pregnant_as1):
    # Filter for delivery records
    deliveries = Only_Pregnant_as1.filter(F.col("delivery_record_only") == 1)

    # Get the first delivery date per person
    first_del = deliveries.groupBy("person_id").agg(
        F.min("date").alias("first_delivery_date")
    )

    print("First delivery sample size:", first_del.count())  # Moved inside the function

    return first_del  # Correct indentation


### addition of the columns

In [ ]:
# Purpose: The purpose of this table is to merge boh tables that were filtered down and bring the first delivery date into the table.
# Creator: for the Team1 project
# Created/Updated 04/03/2025
# Description:



from pyspark.sql import functions as F

def merging(DDateSetup, Only_Pregnant_as1):

    merged_df = DDateSetup.join(
        Only_Pregnant_as1,
        on="person_id",
        how="left"
    )

    return merged_df


### Adding the Delivery flags of 1 year prior, post and 280 days prior as sensitivity test


In [ ]:
# Adding the flag to all patients for the delivery 1 year prior 1 yer post and 280 prior

# This will allow us to sort patients that had only these instances delivery pre post and clinical diagnosis of any concepts that we have interst in.
#for EHR data it comes as a clustered toogether concepts and can be segregated on this basis. To indentify Delivery date for these columns in our consepts


def All_PATIENTS_FLAG(merging):


    df = merging

# Create delivery 1 year prior and 1 year post dates from first_delivery_date plus additional 280 days prior

    df = df.withColumn("date_1y_prior_1st_delivery", F.add_months(F.col("first_delivery_date"), -12))
    df = df.withColumn("date_1y_after_1st_delivery", F.add_months(F.col("first_delivery_date"), +12))
    df = df.withColumn("date_280d_prior_1st_delivery", F.date_add(F.col("first_delivery_date"), -280))


    # To sate delivery date for these columns in our conceptsets that are clustured toogether in the excel (table content) if something is missing willnot be covred in these groups

    all_columns = df.columns
    start_col = "delivery_mode_caesarean"
    end_col = "patient_death_at_visit"

    start_idx = all_columns.index(start_col)
    end_idx = all_columns.index(end_col)
    dv_columns = all_columns[start_idx:end_idx + 1]


    # Add *_flag_1y and *_flag_280d columns per DV

    for col_name in dv_columns:

        # Flag using 1-year prior
        flag_col_1y = f"{col_name}_flag_1y"

        df = df.withColumn(
            flag_col_1y,
            F.when(F.col(col_name) == 0, "no")
             .when(
                (F.col(col_name) == 1) &
                (F.col("date") < F.col("date_1y_prior_1st_delivery")),
                "before_1y_prior_to_1st_delivery")
             .when(
                (F.col(col_name) == 1) &
                (F.col("date") >= F.col("date_1y_prior_1st_delivery")) &
                (F.col("date") < F.col("first_delivery_date")),
                "during_1y_prior_to_1st_delivery")
             .when(
                (F.col(col_name) == 1) &
                (F.col("date") >= F.col("first_delivery_date")) &
                (F.col("date") <= F.col("date_1y_after_1st_delivery")),
                "during_1y_postpartum")
             .otherwise("undefined")
        )

        # Flag using 280-day prior
        flag_col_280d = f"{col_name}_flag_280d"

        df = df.withColumn(
            flag_col_280d,
            F.when(F.col(col_name) == 0, "no")
             .when(
                (F.col(col_name) == 1) &
                (F.col("date") < F.col("date_280d_prior_1st_delivery")),
                "before_280d_prior_to_1st_delivery")
             .when(
                (F.col(col_name) == 1) &
                (F.col("date") >= F.col("date_280d_prior_1st_delivery")) &
                (F.col("date") < F.col("first_delivery_date")),
                "during_280d_prior_to_1st_delivery")
             .when(
                (F.col(col_name) == 1) &
                (F.col("date") >= F.col("first_delivery_date")) &
                (F.col("date") < F.col("date_1y_after_1st_delivery")),
                "during_1y_postpartum")
             .otherwise("undefined")
        )


    # Create dummy variables for all *_flag columns

    flag_cols = [col for col in df.columns if col.endswith("_flag_1y") or col.endswith("_flag_280d")]

    for flag_col in flag_cols:
        unique_values = df.select(flag_col).distinct().rdd.flatMap(lambda x: x).collect()

        for value in unique_values:
            if value is not None:
                safe_value = value.replace(" ", "_")
                dummy_col_name = f"{flag_col}_{safe_value}"
                df = df.withColumn(
                    dummy_col_name,
                    F.when(F.col(flag_col) == value, 1).otherwise(0)
                )


    # Drop dummy columns for "no" and "undefined"

    drop_cols = [col for col in df.columns if col.endswith("_flag_1y_no") or col.endswith("_flag_1y_undefined") or col.endswith("_flag_280d_no") or col.endswith("_flag_280d_undefined")]
    df = df.drop(*drop_cols)


    # Return final DataFrame with flags and dummies

    return df


# Global imports and functions included below

from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, LongType, DoubleType



In [ ]:
#additional changes in the code
# Adding the flag to all patients for the delivery 1 year prior 1 yer post and 280 prior

# This will allow us to sort patients that had only these instances delivery pre post and clinical diagnosis of any concepts that we have interst in.
#for EHR data it comes as a clustered toogether concepts and can be segregated on this basis. To indentify Delivery date for these columns in our consepts
# Taging the columns uing year prior and 280 days prior
# Preparing data for the model using onehot encoding for yes or no 1 and 0


def All_PATIENTS_FLAG(merging):


    df = merging

# Create delivery 1 year prior and 1 year post dates from first_delivery_date plus additional 280 days prior

    df = df.withColumn("_1y_prior_1st_delivery", F.add_months(F.col("first_delivery_date"), -12))
    df = df.withColumn("_1y_after_1st_delivery", F.add_months(F.col("first_delivery_date"), +12))
    df = df.withColumn("_280d_prior_1st_delivery", F.date_add(F.col("first_delivery_date"), -280))
# To sate delivery date for these columns in our conceptsets that are clustured toogether in the excel (table content) if something is missing willnot be covred in these groups
    all_columns = df.columns
    start_col = "delivery_mode_caesarean"
    end_col = "patient_death_at_visit"

    start_idx = all_columns.index(start_col)
    end_idx = all_columns.index(end_col)
    delv_columns = all_columns[start_idx:end_idx + 1]


# Add *_flag_1y and *_flag_280d columns per Delivery

    for col_name in delv_columns:

# Flag using 1-year prior
        flag_col_1y = f"{col_name}_flag_1y"

        df = df.withColumn(
            flag_col_1y,
            F.when(F.col(col_name) == 0, "no")
             .when(
                (F.col(col_name) == 1) &
                (F.col("date") < F.col("_1y_prior_1st_delivery")),
                "before_1y_prior_to_1st_delivery")
             .when(
                (F.col(col_name) == 1) &
                (F.col("date") >= F.col("_1y_prior_1st_delivery")) &
                (F.col("date") < F.col("first_delivery_date")),
                "during_1y_prior_to_1st_delivery")
             .when(
                (F.col(col_name) == 1) &
                (F.col("date") >= F.col("first_delivery_date")) &
                (F.col("date") <= F.col("_1y_after_1st_delivery")),
                "during_1y_postpartum")
             .otherwise("undefined")
        )
# Flag using 280-day prior
        flag_col_280d = f"{col_name}_flag_280d"

        df = df.withColumn(
            flag_col_280d,
            F.when(F.col(col_name) == 0, "no")
             .when(
                (F.col(col_name) == 1) &
                (F.col("date") < F.col("_280d_prior_1st_delivery")),
                "before_280d_prior_to_1st_delivery")
             .when(
                (F.col(col_name) == 1) &
                (F.col("date") >= F.col("_280d_prior_1st_delivery")) &
                (F.col("date") < F.col("first_delivery_date")),
                "during_280d_prior_to_1st_delivery")
             .when(
                (F.col(col_name) == 1) &
                (F.col("date") >= F.col("first_delivery_date")) &
                (F.col("date") < F.col("_1y_after_1st_delivery")),
                "during_1y_postpartum")
             .otherwise("undefined")
        )
# Create one hot encoding variables for all *_flag columns

    flag_cols = [col for col in df.columns if col.endswith("_flag_1y") or col.endswith("_flag_280d")]

    for flag_col in flag_cols:
        unique_values = df.select(flag_col).distinct().rdd.flatMap(lambda x: x).collect()

        for value in unique_values:
            if value is not None:
                safe_value = value.replace(" ", "_")
                dummy_col_name = f"{flag_col}_{safe_value}"
                df = df.withColumn(
                    dummy_col_name,
                    F.when(F.col(flag_col) == value, 1).otherwise(0)
                )


# Drop one hot encoding columns for "no" and "undefined"

    drop_cols = [col for col in df.columns if col.endswith("_flag_1y_no") or col.endswith("_flag_1y_undefined") or col.endswith("_flag_280d_no") or col.endswith("_flag_280d_undefined")]
    df = df.drop(*drop_cols)


# Return final DataFrame with flags and dummies

    return df

#Global imports and functions included below

from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, LongType, DoubleType




## Continuing the code with adding another one hot encodign into the variables

In [ ]:
# Purpose: This function ALL_PATIENTS_SUMMARY performs aggregation and summarization of one-hot encoded patient features from a Spark DataFrame for the Team1 project.
# It processes binary event flags (*_flag_1y_*, *_flag_280d_*), computes summary indicators, and joins with demographic and death data for adult female patients.
# Creator: for Team1 project

# description: Step-by-step Summary: #1. Input DataFrames:

# All_PATIENTS_FLAG: Contains one-hot encoded clinical/event flags.

# everyone_cohort_de_id: Base cohort with patient demographics.

# person: Unused in this function.

# everyone_patient_deaths: Contains death status per patient.

## 2. Group One-Hot Encoded Columns:

# Uses _flag_1y_ and _flag_280d_ patterns to group similar events.

# 3. Aggregate Flags:

# Converts each group of flags into a single summary column (e.g., *_indicator) using max() to indicate whether the event occurred.

# 4. Add Extra Features:

# If available, includes max BMI and vaccine dose count.

# 5. Aggregate Data by Patient:

# Groups by person_id and applies aggregation.

# 6. Join Death Data:

# Merges patient death indicator.

# 7. Filter Adult Females:

# Restricts dataset to female patients aged 18+.



def ALL_PATIENTS_SUMMARY(All_PATIENTS_FLAG,everyone_cohort_de_id,person, everyone_patient_deaths):

    df = All_PATIENTS_FLAG
    deaths_df = everyone_patient_deaths.select('person_id', 'patient_death')


# Group all *_flag_1y_* and *_flag_280d_* one hot encoded columns

    flag_groups = defaultdict(list)

    for col in df.columns:
        if "_flag_1y_" in col:
            dv = col.split("_flag_1y_")[0]
            time = col.split("_flag_1y_")[1]
            flag_groups[f"{dv}_flag_1y"].append((col, time))
        elif "_flag_280d_" in col:
            dv = col.split("_flag_280d_")[0]
            time = col.split("_flag_280d_")[1]
            flag_groups[f"{dv}_flag_280d"].append((col, time))


# Aggregate one hot encoding columns to create *_indicator columns

    agg_expressions = []

    for dv_flag, cols in flag_groups.items():
        for col_name, timing in cols:
            suffix = f"{dv_flag.replace('_flag_', '_')}_{timing}_indicator"
            agg_expressions.append(F.max(col_name).alias(suffix))


# Additional columns (BMI, vaccine)

    if "BMI_rounded" in df.columns:
        agg_expressions.append(F.max("BMI_rounded").alias("BMI_max_observed_or_calculated"))
    if "had_vaccine_administered" in df.columns:
        agg_expressions.append(F.sum("had_vaccine_administered").alias("total_number_of_COVID_vaccine_doses"))


# Run final aggregation

    df = df.groupby("person_id").agg(*agg_expressions)


# Join death info

    df = df.join(deaths_df, 'person_id', 'left').withColumnRenamed('patient_death', 'patient_death_indicator')


# Join to cohort base and restrict to adults

    df = everyone_cohort_de_id.join(df, 'person_id', 'right')
    df = df.filter((F.col("age") >= 18) & (F.col("sex") == "FEMALE"))


# Print unique patient count

    unique_patient_count = df.select("person_id").distinct().count()
    print(f"Number of unique patients: {unique_patient_count}")

    return df


# Global imports and functions included below

from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, LongType, DoubleType
from collections import defaultdict
from pyspark.sql.types import IntegerType, LongType, DoubleType




Final table grouping the columns and so on

In [ ]:
# Purpose: Generate a cleaned and finalized patient history summary table focused on delivery-related outcomes and comorbidities.
#
# Creator: [Team1]
# Last Updated: [2025-04-20]
#
# Description:
#   This function takes the `ALL_PATIENTS_SUMMARY` DataFrame and selects relevant demographic and clinical features
#   including depression, preeclampsia, C-section history, and COVID diagnosis indicators. It creates flags for
#   postpartum outcomes while excluding patients with a history of the condition prior to their first delivery
#   (based on 1-year and 280-day time windows). This helps isolate incident (new) cases occurring postpartum.
#
#   The returned DataFrame is suitable for downstream analysis such as modeling or statistical comparisons.


def Final_history_table(ALL_PATIENTS_SUMMARY):
    df = ALL_PATIENTS_SUMMARY

    df_selected = df.select(
        "person_id", "total_visits", "sex", "age", "race", "race_ethnicity", "data_partner_id",
        # Depression
        F.col("depression_1y_during_1y_postpartum_indicator").alias("postpartum_depression"),
        F.col("depression_1y_during_1y_prior_to_1st_delivery_indicator").alias("depression_hist_1y_prior"),
        F.col("depression_280d_during_280d_prior_to_1st_delivery_indicator").alias("depression_hist_280d_prior"),

        # Preeclampsia
        ## F.col("preeclampsia_1y_during_1y_postpartum_indicator").alias("postpartum_preeclampsia"), ## Not need it
        F.col("preeclampsia_1y_during_1y_prior_to_1st_delivery_indicator").alias("preeclampsia_hist_1y_prior"),
        F.col("preeclampsia_280d_during_280d_prior_to_1st_delivery_indicator").alias("preeclampsia_hist_280d_prior"),

        # C-section # Like depression
        F.col("delivery_mode_caesarean_1y_during_1y_postpartum_indicator").alias("postpartum_csection"),
        F.col("delivery_mode_caesarean_1y_during_1y_prior_to_1st_delivery_indicator").alias("csection_hist_1y_prior"),
        F.col("delivery_mode_caesarean_280d_during_280d_prior_to_1st_delivery_indicator").alias("csection_hist_280d_prior"),

        # Covid diagnosis indicator
        F.col("LL_COVID_diagnosis_1y_during_1y_prior_to_1st_delivery_indicator").alias("covid_diag_hist_1y_prior"),
        F.col("LL_COVID_diagnosis_280d_during_280d_prior_to_1st_delivery_indicator").alias("covid_diag_hist_280d_prior")

    )

    # Create postpartum_depression_main: NA if history of prior depression, else keep postpartum_depression
    df_selected = df_selected.withColumn(
        "postpartum_depression_main",
        F.when(F.col("depression_hist_1y_prior") == 1, F.lit(None)).otherwise(F.col("postpartum_depression"))
    )

    df_selected = df_selected.withColumn(
        "postpartum_csection_main",
        F.when(F.col("csection_hist_1y_prior") == 1, F.lit(None)).otherwise(F.col("postpartum_csection"))
    )

    df_selected = df_selected.withColumn(
        "postpartum_depression_sens",
        F.when(F.col("depression_hist_280d_prior") == 1, F.lit(None)).otherwise(F.col("postpartum_depression"))
    )

    df_selected = df_selected.withColumn(
        "postpartum_csection_sens",
        F.when(F.col("csection_hist_280d_prior") == 1, F.lit(None)).otherwise(F.col("postpartum_csection"))
    )

    return df_selected



# Global imports and functions included below

from pyspark.sql import functions as F


One hot encoding the race

In [ ]:
from pyspark.sql import functions as F


def feature_engineering(Final_history_table):
    # Step 1: Select necessary columns
    selected_columns = [
        "person_id", "total_visits", "sex", "age", "race", "race_ethnicity", "data_partner_id",
        "postpartum_depression", "depression_hist_1y_prior", "depression_hist_280d_prior",
        "preeclampsia_hist_1y_prior", "preeclampsia_hist_280d_prior", "postpartum_csection",
        "csection_hist_1y_prior", "csection_hist_280d_prior", "covid_diag_hist_1y_prior",
        "covid_diag_hist_280d_prior", "postpartum_depression_main", "postpartum_csection_main",
        "postpartum_depression_sens", "postpartum_csection_sens"
    ]
    df = Final_history_table.select(*selected_columns)

    # Step 2: Replace value in the race column
    replace_dict = {
    'Black or African American': 'Black_or_African_American',
    'Hispanic or Latino': 'Other',
    'Native Hawaiian or Other Pacific Islander': 'Other',
    'American Indian or Alaska Native': 'Other'
    }

    df = df.replace(replace_dict, subset=["race"])

    # Step 3: One-hot encode 'sex' column
    gender_categories = df.select('sex').distinct().rdd.flatMap(lambda x: x).collect()
    gender_exprs = [F.when(F.col('sex') == cat, 1).otherwise(0).alias(f"sex_{cat}") for cat in gender_categories]

    # Step 4: One-hot encode 'race' column
    race_categories = df.select('race').distinct().rdd.flatMap(lambda x: x).collect()
    race_exprs = [F.when(F.col('race') == cat, 1).otherwise(0).alias(f"race_{cat}") for cat in race_categories]

    # Step 5: Combine original features with one-hot encoded columns
    df = df.select(*df.columns, *gender_exprs, *race_exprs)

    # Step 6: Drop original categorical columns and optional categories (to avoid multicollinearity)
    cols_to_drop = ["race"]
    optional_drop = ["race_Unknown"]  # Drop one value per set for dummy variable trap
    for col in optional_drop:
        if col in df.columns:
            cols_to_drop.append(col)
    df = df.drop(*cols_to_drop)

    return df


Summary of the Indicators that were taken into account

In [ ]:

        #Baseline
        "person_id", "total_visits", "sex", "age", "race", "race_ethnicity", "data_partner_id",
        "postpartum_depression", "depression_hist_1y_prior", "depression_hist_280d_prior",
        "preeclampsia_hist_1y_prior", "preeclampsia_hist_280d_prior", "postpartum_csection",
        "csection_hist_1y_prior", "csection_hist_280d_prior", "covid_diag_hist_1y_prior",
        "covid_diag_hist_280d_prior", "postpartum_depression_main", "postpartum_csection_main",
        "postpartum_depression_sens", "postpartum_csection_sens"


        #Other indicators
        "ectopic_pregnancy_1y_during_1y_prior_to_1st_delivery_indicator",
        "tobacco_smoker_1y_during_1y_prior_to_1st_delivery_indicator",
        "tobacco_smoker_1y_before_1y_prior_to_1st_delivery_indicator",
        "tobacco_smoker_280d_before_280d_prior_to_1st_delivery_indicator",
        "tobacco_smoker_280d_during_280d_prior_to_1st_delivery_indicator",
        "LL_COVID_diagnosis_1y_during_1y_prior_to_1st_delivery_indicator",
        "LL_COVID_diagnosis_1y_before_1y_prior_to_1st_delivery_indicator",
        "LL_COVID_diagnosis_1y_during_1y_postpartum_indicator",
        "LL_COVID_diagnosis_280d_before_280d_prior_to_1st_delivery_indicator",
        "LL_COVID_diagnosis_280d_during_280d_prior_to_1st_delivery_indicator",
        "LL_COVID_diagnosis_280d_during_1y_postpartum_indicator",
        "chronic_lung_disease_1y_during_1y_prior_to_1st_delivery_indicator",
        "chronic_lung_disease_1y_before_1y_prior_to_1st_delivery_indicator",
        "chronic_lung_disease_280d_before_280d_prior_to_1st_delivery_indicator",
        "chronic_lung_disease_280d_during_280d_prior_to_1st_delivery_indicator",
        "eclampsia_1y_during_1y_prior_to_1st_delivery_indicator",
        "eclampsia_1y_before_1y_prior_to_1st_delivery_indicator",
        "eclampsia_280d_before_280d_prior_to_1st_delivery_indicator",
        "eclampsia_280d_during_280d_prior_to_1st_delivery_indicator",
        "hypertension_1y_during_1y_prior_to_1st_delivery_indicator",
        "hypertension_1y_before_1y_prior_to_1st_delivery_indicator",
        "hypertension_280d_before_280d_prior_to_1st_delivery_indicator",
        "hypertension_280d_during_280d_prior_to_1st_delivery_indicator",
        "maternal_obesity_1y_during_1y_prior_to_1st_delivery_indicator",
        "maternal_obesity_1y_before_1y_prior_to_1st_delivery_indicator",
        "maternal_obesity_280d_before_280d_prior_to_1st_delivery_indicator",
        "maternal_obesity_280d_during_280d_prior_to_1st_delivery_indicator",
        "preeclampsia_1y_during_1y_prior_to_1st_delivery_indicator",
        "preeclampsia_1y_before_1y_prior_to_1st_delivery_indicator",
        "preeclampsia_280d_before_280d_prior_to_1st_delivery_indicator",
        "preeclampsia_280d_during_280d_prior_to_1st_delivery_indicator",
        "psychosis_1y_during_1y_prior_to_1st_delivery_indicator",
        "psychosis_1y_before_1y_prior_to_1st_delivery_indicator",
        "psychosis_1y_during_1y_postpartum_indicator",
        "psychosis_280d_before_280d_prior_to_1st_delivery_indicator",
        "psychosis_280d_during_280d_prior_to_1st_delivery_indicator",
        "psychosis_280d_during_1y_postpartum_indicator",
        "OBESITY_1y_during_1y_prior_to_1st_delivery_indicator",
        "OBESITY_1y_before_1y_prior_to_1st_delivery_indicator",
        "OBESITY_280d_before_280d_prior_to_1st_delivery_indicator",
        "total_number_of_COVID_vaccine_doses",

## Defining the simple statistics of the groups to do descriptive analysis statistcs: in pythion and R code for the comparison


Paired t test

In [ ]:
#The code performs a paired Ttest to know if there's a significant difference in systolic blood pressure before and after 6 weeks of treatment with Drug X in patients with hypertension"
# Looking at postpartum_year after depression and History 280 days Before
# This is R code

# Define paired t-test function
Paired_t_TestPostpartum_280DaysBefore <- function(Final_history_table) {

  # Perform the paired samples t-test
  t_test_result <- t.test(Final_history_table$postpartum_depression, Final_history_table$depression_hist_280d_prior , paired = TRUE)

   # Print the result of the t-test
  print(t_test_result)

   # Return the t-test result for further use
  return(NULL)

}

#RESULT INTERPRETATION
# T test 92 	This is the t-statistic. A high number = a strong difference between the two groups.
# 	On average, postpartum depression is 3.92% higher than depression during pregnancy.
# This is a very significant result.

#  following the idea:
# You have two groups of weights.

# Within each group, the weights are clustered tightly (small standard deviation).

# But the average weight between the two groups is quite different.

#RESULT: Because the groups are internally consistent (not spread out)...

# And the difference between their means is big...

# That gives us a large t-statistic → which means the difference is statistically significant!



#IN Class Example:

#Key Components of the Output:

#t-value:

#t = -41.468: This is the calculated test statistic, which is quite large in absolute value. The negative sign indicates that the After values are smaller than the Before values on average. This suggests a strong decrease in whatever measurement you're testing (e.g., weight, blood pressure, etc.).

#Degrees of Freedom (df):

#df = 299: The degrees of freedom for the test, which is calculated as n - 1, where n is the number of pairs. Since you have 300 pairs of data, the degrees of freedom is 299.

#p-value:

#p-value < 2.2e-16: This is extremely small, much less than the typical significance level of 0.05. This means there is a very strong statistical significance, and we can confidently reject the null hypothesis (which states that the mean difference between Before and After is 0). Therefore, there is a significant difference between the Before and After measurements.

#Alternative Hypothesis:

#The alternative hypothesis being tested is that the true mean difference between the Before and After measurements is not equal to 0 (i.e., the two means are different).

#95% Confidence Interval:

#Confidence interval: The 95% confidence interval for the mean difference is between -53.62 and -48.76. Since the entire interval is negative and does not contain 0, this further supports the conclusion that the After measurements are significantly smaller than the Before measurements.

#Mean Difference:

#mean difference = -51.19333: The average difference between the Before and After measurements is about -51.19. This indicates that, on average, the value of After is 51.19 units smaller than Before. This suggests a substantial decrease in the measurements from Before to After.

#Interpretation:
#The large t-value (-41.468) and very small p-value (< 2.2e-16) suggest a highly significant result, indicating that the difference between the two groups (Before and After) is statistically significant.

#The mean difference of -51.19 suggests that whatever variable you’re measuring (e.g., weight, blood pressure, etc.) has decreased on average by about 51.19 units after the intervention or treatment.

#The 95% confidence interval for the mean difference is (-53.62, -48.76), which tells you that the true mean difference between the Before and After measurements lies within this range with 95% confidence. Since the entire confidence interval is negative and does not include 0, it further supports the conclusion of a significant decrease.

#Conclusion:
#Based on the results of the paired t-test:

#There is a statistically significant decrease in the measurement from Before to After.

#The mean difference is approximately -51.19, meaning the After measurement is lower than the Before measurement by that amount.

#The p-value is extremely small (less than 2.2e-16), indicating very strong evidence against the null hypothesis that there is no difference between the two time points.


In [ ]:
# Ttest python

import pandas as pd
import scipy.stats as stats
from pyspark.sql import SparkSession

def Python_age_ttest(Final_history_table):

    """
    Perform a one-sample t-test (greater) comparing the mean of 'age' to 35 using Spark DataFrame.

    Parameters:
    Final (DataFrame): A PySpark DataFrame with an 'age' column.

    Returns:
    None
    """
    # Convert Spark DataFrame to Pandas DataFrame
    final_pd = Final_history_table.select("age").toPandas()

    # Perform the one-sample t-test
    sample = final_pd['age']
    t_statistic, p_value_two_sided = stats.ttest_1samp(sample, popmean=35)

    # Convert two-sided p-value to one-sided (greater)
    if t_statistic > 0:
        p_value_one_sided = p_value_two_sided / 2
    else:
        p_value_one_sided = 1 - (p_value_two_sided / 2)

    # Print the results
    print("T-statistic:", t_statistic)
    print("One-sided p-value (greater):", p_value_one_sided)


In [ ]:
# R t-test
RprogamT_test_Age35 <- function(Final_history_table) {

    #the function compares the mean of age to 35.
  t_test_result <- t.test(Final_history_table$age, mu = 35, alternative="greater")


  # Print the result of the t-test
  print(t_test_result)

}

In [ ]:
# paied t test python
# Working on this one

from pyspark.sql import functions as F
from scipy.stats import t

def paired_t_test_postpartum_1y(Final_history_table):
    """
    Perform a paired t-test in PySpark between 'postpartum_depression' and
    'depression_hist_1y_prior' without converting to Pandas.

    Parameters:
    - Final_history_table: PySpark DataFrame

    Prints:
    - T-statistic, sample size, mean difference, and p-value approximation
    """

    # Step 1: Select and drop missing data
    df_filtered = Final_history_table.select(
        "postpartum_depression",
        "depression_hist_1y_prior"
    ).dropna(subset=["postpartum_depression", "depression_hist_1y_prior"])

    # Step 2: Compute difference
    df_diff = df_filtered.withColumn(
        "diff",
        F.col("postpartum_depression") - F.col("depression_hist_1y_prior")
    )

    # Step 3: Calculate statistics
    stats = df_diff.select(
        F.count("diff").alias("n"),
        F.mean("diff").alias("mean_diff"),
        F.stddev("diff").alias("std_diff")
    ).collect()[0]

    n = stats["n"]
    mean_diff = stats["mean_diff"]
    std_diff = stats["std_diff"]

    # Step 4: Calculate t-statistic
    t_stat = mean_diff / (std_diff / (n ** 0.5)) if std_diff else float('inf')

    # Step 5: Approximate two-sided p-value
    p_value = 2 * (1 - t.cdf(abs(t_stat), df=n-1))

    # Output
    print("\nPaired t-test between postpartum_depression and depression_hist_1y_prior:")
    print(f"Sample size (n): {n}")
    print(f"Mean difference: {mean_diff:.8f}")
    print(f"Standard deviation of differences: {std_diff:.8f}")
    print(f"T-statistic: {t_stat:.3f}")
    print(f"Approx. P-value (two-sided): {p_value:.3g}")


In [ ]:

#The code performs a paired Ttest to know if there's a significant difference in systolic blood pressure before and after 6 weeks of treatment with Drug X in patients with hypertension"
# Looking at postpartum_year after depression and Historry Year Before

# Define paired t-test function
Paired_t_TestPostpartum_1YearBefore <- function(Final_history_table) {

  # Perform the paired samples t-test
  t_test_result <- t.test(Final_history_table$postpartum_depression, Final_history_table$depression_hist_1y_prior, paired = TRUE)

   # Print the result of the t-test
  print(t_test_result)

   # Return the t-test result for further use
  return(NULL)

}


In [ ]:
# Chi test

from pyspark.sql import functions as F
from scipy.stats import chisquare

from pyspark.sql.functions import col


def chi_square_postpartum(Final_history_table):

    # Count occurrences of each value in postpartum_depression (assumed to be binary: 0/1)
    counts_df = Final_history_table.groupBy("postpartum_depression").count()

    # Collect the results to the driver
    counts = counts_df.orderBy("postpartum_depression").select("count").rdd.flatMap(lambda x: x).collect()

    # Check if there are exactly 2 categories (0 and 1)
    if len(counts) != 2:
        print("There must be exactly two categories (e.g., 0 and 1) in 'postpartum_depression'.")
        return

    # Expected frequencies for a 50/50 distribution
    total = sum(counts)
    expected = [total / 2, total / 2]

    # Run Chi-Square Goodness-of-Fit test
    chi_stat, p_value = chisquare(counts, f_exp=expected)

    # Print results
    print(f"Observed counts (postpartum_depression): {counts}")
    print(f"Expected counts (50/50): {expected}")
    print("Chi-square statistic:", chi_stat)
    print("P-value:", p_value)



In [ ]:
#chi test r
# Results:
# The number of people with and without postpartum depression is not close to 50:50. It's very unbalanced, and that difference is statistically significant.


# Define the Chi-Square Goodness of Fit function with a 50:50 expected distribution
Chi_Square_Goodness_50_50_R <- function(Final_history_table) {

  # Check if the 'postpartum_depression' variable exists in the dataset
  if (!"postpartum_depression" %in% colnames(Final_history_table)) {
    stop("The 'postpartum_depression' variable does not exist in the dataset.")
  }

  # Creating observed frequencies for 'postpartum_depression' in the dataset
  observed <- table(Final_history_table$postpartum_depression)  # Count occurrences of each level

  # Ensure the observed data has exactly two categories (e.g., 0 and 1)
  if (length(observed) != 2) {
    stop("There must be exactly two levels in the 'postpartum_depression' variable.")
  }

  # Expected probabilities for a 50:50 distribution
  expected_probabilities <- c(0.5, 0.5)

  # Perform the Chi-Square Goodness of Fit Test
  chi_square_result <- chisq.test(observed, p = expected_probabilities)

  # Print the result of the Chi-Square test
  print(chi_square_result)

  # Return the Chi-Square result for further use
  return(Final_history_table)
}


In [ ]:
# Mann Wihtney R
#The code performs a two-sample-T-test comparing the mean of weight between participants using the Wilcoxon rank-sum test (also known as Mann-Whitney U test) (This is a Non-Parametric Tests)

# Define Mann_Whitneyfunction
Mann_Whitney_Test <- function(Final_history_table) {

# Perform the Mann-Whitney U test (Wilcoxon rank sum test)
    mann_whitney_test_result <- wilcox.test(age ~ postpartum_depression, data = Final_history_table)

 # Print the test result
     print(mann_whitney_test_result)

# Return the test result
     return(Final_history_table)

}

Featrue Engineering

In [ ]:

from pyspark.sql import functions as F


def Feature_Eng_5( unnamed_7):
    # Step 1: Select necessary columns
    selected_columns = [
        "person_id", "total_visits", "sex", "age", "race", "race_ethnicity", "data_partner_id",
        "postpartum_depression", "depression_hist_1y_prior", "depression_hist_280d_prior",
        "preeclampsia_hist_1y_prior", "preeclampsia_hist_280d_prior", "postpartum_csection",
        "csection_hist_1y_prior", "csection_hist_280d_prior", "covid_diag_hist_1y_prior",
        "covid_diag_hist_280d_prior", "postpartum_depression_main", "postpartum_csection_main",
        "postpartum_depression_sens", "postpartum_csection_sens",
        #New Indicators
        "eclampsia_any_1y_prior_to_1st_delivery","eclampsia_any_before__1y_prior_to_1st_delivery",
        "eclampsia_any_280d_before_prior_to_1st_delivery","eclampsia_any_280d_during_prior_to_1st_delivery",

        "ectopic_pregnancy_1y_during_1y_prior_to_1st_delivery_indicator",
        "tobacco_smoker_1y_during_1y_prior_to_1st_delivery_indicator",
        "tobacco_smoker_1y_before_1y_prior_to_1st_delivery_indicator",
        "tobacco_smoker_280d_before_280d_prior_to_1st_delivery_indicator",
        "tobacco_smoker_280d_during_280d_prior_to_1st_delivery_indicator",
        "LL_COVID_diagnosis_1y_during_1y_prior_to_1st_delivery_indicator",
        "LL_COVID_diagnosis_1y_before_1y_prior_to_1st_delivery_indicator",
        "LL_COVID_diagnosis_1y_during_1y_postpartum_indicator",
        "LL_COVID_diagnosis_280d_before_280d_prior_to_1st_delivery_indicator",
        "LL_COVID_diagnosis_280d_during_280d_prior_to_1st_delivery_indicator",
        "LL_COVID_diagnosis_280d_during_1y_postpartum_indicator",
        "chronic_lung_disease_1y_during_1y_prior_to_1st_delivery_indicator",
        "chronic_lung_disease_1y_before_1y_prior_to_1st_delivery_indicator",
        "chronic_lung_disease_280d_before_280d_prior_to_1st_delivery_indicator",
        "chronic_lung_disease_280d_during_280d_prior_to_1st_delivery_indicator",

        "hypertension_1y_during_1y_prior_to_1st_delivery_indicator",
        "hypertension_1y_before_1y_prior_to_1st_delivery_indicator",
        "hypertension_280d_before_280d_prior_to_1st_delivery_indicator",
        "hypertension_280d_during_280d_prior_to_1st_delivery_indicator",
        "maternal_obesity_1y_during_1y_prior_to_1st_delivery_indicator",
        "maternal_obesity_1y_before_1y_prior_to_1st_delivery_indicator",
        "maternal_obesity_280d_before_280d_prior_to_1st_delivery_indicator",
        "maternal_obesity_280d_during_280d_prior_to_1st_delivery_indicator",

        "psychosis_1y_during_1y_prior_to_1st_delivery_indicator",
        "psychosis_1y_before_1y_prior_to_1st_delivery_indicator",
        "psychosis_1y_during_1y_postpartum_indicator",
        "psychosis_280d_before_280d_prior_to_1st_delivery_indicator",
        "psychosis_280d_during_280d_prior_to_1st_delivery_indicator",
        "psychosis_280d_during_1y_postpartum_indicator",
        #New Indicators
        "diabetes_any_1y_prior_to_1st_delivery","diabetes_any_before__1y_prior_to_1st_delivery",
        "diabetes_any_280d_before_prior_to_1st_delivery","diabetes_any_280d_during_prior_to_1st_delivery",

        "OBESITY_1y_during_1y_prior_to_1st_delivery_indicator",
        "OBESITY_1y_before_1y_prior_to_1st_delivery_indicator",
        "OBESITY_280d_before_280d_prior_to_1st_delivery_indicator",
        "total_number_of_COVID_vaccine_doses"

    ]
    df = unnamed_7.select(*selected_columns)

    # Step 2: Replace value in the race column
    replace_dict = {
    'Black or African American': 'Black_or_African_American',
    'Hispanic or Latino': 'Other',
    'Native Hawaiian or Other Pacific Islander': 'Other',
    'American Indian or Alaska Native': 'Other'
    }

    df = df.replace(replace_dict, subset=["race"])

    # Step 3: One-hot encode 'sex' column
    gender_categories = df.select('sex').distinct().rdd.flatMap(lambda x: x).collect()
    gender_exprs = [F.when(F.col('sex') == cat, 1).otherwise(0).alias(f"sex_{cat}") for cat in gender_categories]

    # Step 4: One-hot encode 'race' column
    race_categories = df.select('race').distinct().rdd.flatMap(lambda x: x).collect()
    race_exprs = [F.when(F.col('race') == cat, 1).otherwise(0).alias(f"race_{cat}") for cat in race_categories]

    # Step 5: Combine original features with one-hot encoded columns
    df = df.select(*df.columns, *gender_exprs, *race_exprs)

    # Step 6: Drop original categorical columns and optional categories (to avoid multicollinearity)
    cols_to_drop = ["race"]
    optional_drop = ["race_Unknown"]  # Drop one value per set for dummy variable trap
    for col in optional_drop:
        if col in df.columns:
            cols_to_drop.append(col)
    df = df.drop(*cols_to_drop)

    return df



In [ ]:
# This code it to try the coding of one hot encodding and it is on the old node that has only selected columns

from pyspark.sql import functions as F

def feature_engineering2(Final_history_table):
    # Step 1: Select necessary columns
    selected_columns = [
        "person_id", "total_visits", "sex", "age", "race", "race_ethnicity", "data_partner_id",
        "postpartum_depression", "depression_hist_1y_prior", "depression_hist_280d_prior",
        "preeclampsia_hist_1y_prior", "preeclampsia_hist_280d_prior", "postpartum_csection",
        "csection_hist_1y_prior", "csection_hist_280d_prior", "covid_diag_hist_1y_prior",
        "covid_diag_hist_280d_prior", "postpartum_depression_main", "postpartum_csection_main",
        "postpartum_depression_sens", "postpartum_csection_sens"
    ]
    df = Final_history_table.select(*selected_columns)

    # Step 2: Replace value in the race column
    replace_dict = {
        'Black or African American': 'Black_or_African_American',
        'Hispanic or Latino': 'Other',
        'Native Hawaiian or Other Pacific Islander': 'Other',
        'American Indian or Alaska Native': 'Other'
    }
    df = df.replace(replace_dict, subset=["race"])

    # Step 3: One-hot encode 'race' column (only)
    race_categories = df.select('race').distinct().rdd.flatMap(lambda x: x).collect()
    race_exprs = [F.when(F.col('race') == cat, 1).otherwise(0).alias(f"race_{cat}") for cat in race_categories]

    # Step 4: Combine original features with one-hot encoded columns
    df = df.select(*df.columns, *race_exprs)

    # Step 5: Drop original categorical columns and optional categories (to avoid multicollinearity)
    cols_to_drop = ["race"]
    optional_drop = ["race_Unknown"]  # Drop one value per set for dummy variable trap
    for col in optional_drop:
        if col in df.columns:
            cols_to_drop.append(col)
    df = df.drop(*cols_to_drop)

    return df


# Doing gender as 1  for the FEMALe

In [ ]:
#

from pyspark.sql import functions as F

def Feature_Eng_1(Data_cleaning_1):

    # Step 1: Keep all columns
    df = Data_cleaning_1

    # Step 2: Replace values in the 'race' column
    replace_dict = {
        'Black or African American': 'Black_or_African_American',
        'Hispanic or Latino': 'Other',
        'Native Hawaiian or Other Pacific Islander': 'Other',
        'American Indian or Alaska Native': 'Other'
    }


    # Step 2.5: Replace 'FEMALE' in 'sex' column with 1
    df = df.withColumn('sex', F.when(F.col('sex') == 'FEMALE', 1).otherwise(F.col('sex')))

    # Step 3: One-hot encode 'race' column (only)
    df = df.replace(replace_dict, subset=["race"])

    # Step 3: One-hot encode 'race' column (only)
    race_categories = df.select('race').distinct().rdd.flatMap(lambda x: x).collect()
    race_exprs = [F.when(F.col('race') == cat, 1).otherwise(0).alias(f"race_{cat}") for cat in race_categories]

    # Step 4: Combine original features with one-hot encoded columns
    df = df.select(*df.columns, *race_exprs)

    # Step 5: Drop original categorical columns and optional categories (to avoid multicollinearity)
    cols_to_drop = ["race"]
    optional_drop = ["race_Unknown"]  # Drop one value per set for variable trap
    for col in optional_drop:
        if col in df.columns:
            cols_to_drop.append(col)
    df = df.drop(*cols_to_drop)

    return df


Code for data cleaning to focuse on age 18 till 50, and gender

In [ ]:
# Purpose: Prepares and transforms patient data by selecting relevant clinical and demographic features,
# standardizing the "race" column, one-hot encoding it (done in a previous node),
# and filtering the cohort to include only patients aged 18 to 50.
# Team1_
# Date: 04_24_25
# Description: Performs feature engineering on a Spark DataFrame by filtering valid age and sex values,
# selecting specific races, and restricting the cohort to patients aged 18 to 50.

def Data_Cleaning_6(Feature_Eng_5):

    selected_columns = [
        "person_id", "total_visits", "sex", "age", "race", "race_ethnicity", "data_partner_id",
        "postpartum_depression", "depression_hist_1y_prior", "depression_hist_280d_prior",
        "preeclampsia_hist_1y_prior", "preeclampsia_hist_280d_prior", "postpartum_csection",
        "csection_hist_1y_prior", "csection_hist_280d_prior", "covid_diag_hist_1y_prior",
        "covid_diag_hist_280d_prior", "postpartum_depression_main", "postpartum_csection_main",
        "postpartum_depression_sens", "postpartum_csection_sens",
        # New Indicators
        "eclampsia_any_1y_prior_to_1st_delivery","eclampsia_any_before__1y_prior_to_1st_delivery",
        "eclampsia_any_280d_before_prior_to_1st_delivery","eclampsia_any_280d_during_prior_to_1st_delivery",
        "ectopic_pregnancy_1y_during_1y_prior_to_1st_delivery_indicator",
        "tobacco_smoker_1y_during_1y_prior_to_1st_delivery_indicator",
        "tobacco_smoker_1y_before_1y_prior_to_1st_delivery_indicator",
        "tobacco_smoker_280d_before_280d_prior_to_1st_delivery_indicator",
        "tobacco_smoker_280d_during_280d_prior_to_1st_delivery_indicator",
        "LL_COVID_diagnosis_1y_during_1y_prior_to_1st_delivery_indicator",
        "LL_COVID_diagnosis_1y_before_1y_prior_to_1st_delivery_indicator",
        "LL_COVID_diagnosis_1y_during_1y_postpartum_indicator",
        "LL_COVID_diagnosis_280d_before_280d_prior_to_1st_delivery_indicator",
        "LL_COVID_diagnosis_280d_during_280d_prior_to_1st_delivery_indicator",
        "LL_COVID_diagnosis_280d_during_1y_postpartum_indicator",
        "chronic_lung_disease_1y_during_1y_prior_to_1st_delivery_indicator",
        "chronic_lung_disease_1y_before_1y_prior_to_1st_delivery_indicator",
        "chronic_lung_disease_280d_before_280d_prior_to_1st_delivery_indicator",
        "chronic_lung_disease_280d_during_280d_prior_to_1st_delivery_indicator",
        "hypertension_1y_during_1y_prior_to_1st_delivery_indicator",
        "hypertension_1y_before_1y_prior_to_1st_delivery_indicator",
        "hypertension_280d_before_280d_prior_to_1st_delivery_indicator",
        "hypertension_280d_during_280d_prior_to_1st_delivery_indicator",
        "maternal_obesity_1y_during_1y_prior_to_1st_delivery_indicator",
        "maternal_obesity_1y_before_1y_prior_to_1st_delivery_indicator",
        "maternal_obesity_280d_before_280d_prior_to_1st_delivery_indicator",
        "maternal_obesity_280d_during_280d_prior_to_1st_delivery_indicator",
        "psychosis_1y_during_1y_prior_to_1st_delivery_indicator",
        "psychosis_1y_before_1y_prior_to_1st_delivery_indicator",
        "psychosis_1y_during_1y_postpartum_indicator",
        "psychosis_280d_before_280d_prior_to_1st_delivery_indicator",
        "psychosis_280d_during_280d_prior_to_1st_delivery_indicator",
        "psychosis_280d_during_1y_postpartum_indicator",
        "diabetes_any_1y_prior_to_1st_delivery","diabetes_any_before__1y_prior_to_1st_delivery",
        "diabetes_any_280d_before_prior_to_1st_delivery","diabetes_any_280d_during_prior_to_1st_delivery",
        "OBESITY_1y_during_1y_prior_to_1st_delivery_indicator",
        "OBESITY_1y_before_1y_prior_to_1st_delivery_indicator",
        "OBESITY_280d_before_280d_prior_to_1st_delivery_indicator",
        "total_number_of_COVID_vaccine_doses"
    ]

    # Filter for valid age between 18 and 50
    df_age = Feature_Eng_5.filter((Feature_Eng_5.age >= 18) & (Feature_Eng_5.age <= 50))
    print('# of patients aged 18 to 50:', df_age.count())

    # Filter for valid gender
    df_gender = df_age.filter((df_age.sex == 'MALE') | (df_age.sex == 'FEMALE'))
    print('# of patients with gender Male or Female:', df_gender.count())

    # Filter for selected race categories
    df_race = df_gender.filter(
        (df_gender.race == 'White') |
        (df_gender.race == 'Asian') |
        (df_gender.race == 'Black or African American') |
        (df_gender.race == 'Unknown')
    )
    print('# of patients with selected race categories:', df_race.count())

    return df_race


In [ ]:
# Purpose: Prepares and transforms patient data by selecting relevant clinical and demographic features, standardizing the "race" column, one-hot encoding it that was done in the previous node, and filtering the cohort to include only patients aged 18 to 50.
# Team1_
# Date: 04_24_25
# Description: It performs feature engineering on a Spark DataFrame by Filtering the cohort to include only patients aged 18 to 50.


# Purpose: Prepares and transforms patient data by selecting relevant clinical and demographic features,
# and filtering the cohort to include only patients aged 18 to 50.
# Team1_
# Date: 04_24_25
# Description: Performs feature engineering on a Spark DataFrame by filtering on age
# and selecting specific clinical and demographic columns.

def Data_Cleaning_6(Feature_Eng_5):

    selected_columns = [
        "person_id", "total_visits", "sex", "age", "race", "race_ethnicity", "data_partner_id",
        "postpartum_depression", "depression_hist_1y_prior", "depression_hist_280d_prior",
        "preeclampsia_hist_1y_prior", "preeclampsia_hist_280d_prior", "postpartum_csection",
        "csection_hist_1y_prior", "csection_hist_280d_prior", "covid_diag_hist_1y_prior",
        "covid_diag_hist_280d_prior", "postpartum_depression_main", "postpartum_csection_main",
        "postpartum_depression_sens", "postpartum_csection_sens",
        "eclampsia_any_1y_prior_to_1st_delivery","eclampsia_any_before__1y_prior_to_1st_delivery",
        "eclampsia_any_280d_before_prior_to_1st_delivery","eclampsia_any_280d_during_prior_to_1st_delivery",
        "ectopic_pregnancy_1y_during_1y_prior_to_1st_delivery_indicator",
        "tobacco_smoker_1y_during_1y_prior_to_1st_delivery_indicator",
        "tobacco_smoker_1y_before_1y_prior_to_1st_delivery_indicator",
        "tobacco_smoker_280d_before_280d_prior_to_1st_delivery_indicator",
        "tobacco_smoker_280d_during_280d_prior_to_1st_delivery_indicator",
        "LL_COVID_diagnosis_1y_during_1y_prior_to_1st_delivery_indicator",
        "LL_COVID_diagnosis_1y_before_1y_prior_to_1st_delivery_indicator",
        "LL_COVID_diagnosis_1y_during_1y_postpartum_indicator",
        "LL_COVID_diagnosis_280d_before_280d_prior_to_1st_delivery_indicator",
        "LL_COVID_diagnosis_280d_during_280d_prior_to_1st_delivery_indicator",
        "LL_COVID_diagnosis_280d_during_1y_postpartum_indicator",
        "chronic_lung_disease_1y_during_1y_prior_to_1st_delivery_indicator",
        "chronic_lung_disease_1y_before_1y_prior_to_1st_delivery_indicator",
        "chronic_lung_disease_280d_before_280d_prior_to_1st_delivery_indicator",
        "chronic_lung_disease_280d_during_280d_prior_to_1st_delivery_indicator",
        "hypertension_1y_during_1y_prior_to_1st_delivery_indicator",
        "hypertension_1y_before_1y_prior_to_1st_delivery_indicator",
        "hypertension_280d_before_280d_prior_to_1st_delivery_indicator",
        "hypertension_280d_during_280d_prior_to_1st_delivery_indicator",
        "maternal_obesity_1y_during_1y_prior_to_1st_delivery_indicator",
        "maternal_obesity_1y_before_1y_prior_to_1st_delivery_indicator",
        "maternal_obesity_280d_before_280d_prior_to_1st_delivery_indicator",
        "maternal_obesity_280d_during_280d_prior_to_1st_delivery_indicator",
        "psychosis_1y_during_1y_prior_to_1st_delivery_indicator",
        "psychosis_1y_before_1y_prior_to_1st_delivery_indicator",
        "psychosis_1y_during_1y_postpartum_indicator",
        "psychosis_280d_before_280d_prior_to_1st_delivery_indicator",
        "psychosis_280d_during_280d_prior_to_1st_delivery_indicator",
        "psychosis_280d_during_1y_postpartum_indicator",
        "diabetes_any_1y_prior_to_1st_delivery","diabetes_any_before__1y_prior_to_1st_delivery",
        "diabetes_any_280d_before_prior_to_1st_delivery","diabetes_any_280d_during_prior_to_1st_delivery",
        "OBESITY_1y_during_1y_prior_to_1st_delivery_indicator",
        "OBESITY_1y_before_1y_prior_to_1st_delivery_indicator",
        "OBESITY_280d_before_280d_prior_to_1st_delivery_indicator",
        "total_number_of_COVID_vaccine_doses"
    ]

    # Filter for valid age between 18 and 50
    df_age = Feature_Eng_5.filter((Feature_Eng_5.age >= 18) & (Feature_Eng_5.age <= 50))
    print('# of patients aged 18 to 50:', df_age.count())

    # Select only the relevant columns
    df_selected = df_age.select(*selected_columns)

    return df_selected


## Feature Scaling

In [ ]:
# Purpose: Feature Scaling allowed us to scale the continous value age and applies standard scaling.  Then, it concatenates the scaled continuous features back with the categorical features. Finally, it removes quasi-constant features, which have 99% of their #values the same, to ensure the dataset is suitable for model building.
# Date: 04_25_25
# Description: Addoption of the ML_demo_Python code

from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql import functions as F

def Feature_scaling_1(Feature_Eng_1):


    df = Feature_Eng_1
    continuous_features = ["age"]

    # Step 1: Assemble features into a vector
    assembler = VectorAssembler(inputCols=continuous_features, outputCol="cont_features_vec")
    df_vector = assembler.transform(df)

    # Step 2: Apply standard scaler
    scaler = StandardScaler(inputCol="cont_features_vec", outputCol="scaled_features", withMean=True, withStd=True)
    scaler_model = scaler.fit(df_vector)
    df_scaled = scaler_model.transform(df_vector)

    # Step 3: Extract scaled features back to columns (if needed)
    # In Spark, the scaled data stays in vector form. If you want to split it into columns, use this:
    from pyspark.ml.functions import vector_to_array
    df_scaled = df_scaled.withColumn("scaled_age", vector_to_array("scaled_features")[0])

    # Drop intermediate columns
    df_final = df_scaled.drop("cont_features_vec", "scaled_features")

    return df_final



In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql import functions as F

def feature_scaling_2_pyspark(Feature_Eng_1):
    """
    Scales the 'age' column and replaces it with a new 'age_scaled' column in a PySpark DataFrame.

    Parameters:
        Feature_Eng_1 (pyspark.sql.DataFrame): Input DataFrame.

    Returns:
        pyspark.sql.DataFrame: DataFrame with 'age_scaled' instead of 'age'.
    """
    df = Feature_Eng_1

    # Step 1: Assemble 'age' into a feature vector
    assembler = VectorAssembler(inputCols=["age"], outputCol="age_unscaled_vec")
    df = assembler.transform(df)

    # Step 2: Scale 'age' using StandardScaler
    scaler = StandardScaler(inputCol="age_unscaled_vec", outputCol="age_scaled_vec", withMean=True, withStd=True)
    scaler_model = scaler.fit(df)
    df = scaler_model.transform(df)

    # Step 3: Extract the scaled value from the vector into a regular float column
    df = df.withColumn("age_scaled", F.col("age_scaled_vec")[0])

    # Step 4: Drop intermediate columns ('age' and vector columns)
    df = df.drop("age", "age_unscaled_vec", "age_scaled_vec")

    return df


In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler, VarianceThresholdSelector

def feature_scaling_2_pyspark(Feature_Eng_1):
    """
    Scales continuous features and removes quasi-constant features from a PySpark DataFrame.

    Parameters:
        Feature_Eng_1 (pyspark.sql.DataFrame): Input DataFrame with features.

    Returns:
        pyspark.sql.DataFrame: Scaled and cleaned DataFrame.
    """
    df = Feature_Eng_1

    # Continuous feature
    continuous_features = ["age"]

    # Assemble continuous features into a single vector column
    assembler = VectorAssembler(inputCols=continuous_features, outputCol="features_unscaled")
    df = assembler.transform(df)

    # Apply StandardScaler
    scaler = StandardScaler(inputCol="features_unscaled", outputCol="features_scaled", withMean=True, withStd=True)
    scaler_model = scaler.fit(df)
    df = scaler_model.transform(df)

    # Remove quasi-constant features (variance threshold)
    selector = VarianceThresholdSelector(
        varianceThreshold=0.01,
        featuresCol="features_scaled",
        outputCol="features_selected"
    )
    selector_model = selector.fit(df)
    df = selector_model.transform(df)

    # At this point, "features_selected" contains your cleaned features.
    # If you want to split it back into columns, it’s a few more steps. Otherwise you can keep as vector.

    return df.select("features_selected")


Running the logistic regression example

In [ ]:
#The following code builds and evaluates a logistic regression model to predict mortality in patients. It first splits the data into training and testing sets. Then, it trains the #logistic regression model using the training data and evaluates it using the testing data. Various evaluation metrics such as accuracy, F1 score, precision, recall, and AUC-ROC are #calculated to assess the model's performance. A ROC curve is also plotted to visualize the model's performance
# Logistic Regression Model for Mortality Prediction

def logistic_regression_model(Feature_scaling_1):
    df = feature_scaling
    y = df['patient_death_indicator']  # Target variable
    df.drop(['patient_death_indicator'], axis=1, inplace=True)  # Drop target variable from features

    # Split the data into training and testing sets
    train_x, test_x, train_y, test_y = train_test_split(df, y, test_size=0.2, random_state=42)

    # Print the shape of the training and testing sets
    print("Train_x:", train_x.shape)
    print("Test_x:", test_x.shape)
    print("Train_y:", train_y.shape)
    print("Test_y:", test_y.shape)

    # Initialize and train the logistic regression model
    logreg = LogisticRegression(max_iter=1000)
    logreg.fit(train_x, train_y)

    # Evaluate the model's performance
    print("Logistic Regression model Accuracy:", logreg.score(test_x, test_y))
    print("Logistic Regression F1 Score:", f1_score(test_y, logreg.predict(test_x), average='macro'))
    print(classification_report(test_y, logreg.predict(test_x)))

    # Predict probabilities for the test set
    logreg_pred_proba = logreg.predict_proba(test_x)

    # Calculate ROC curve and AUC
    fpr, tpr, thresholds = roc_curve(test_y, logreg_pred_proba[:, 1])
    auc = roc_auc_score(test_y, logreg_pred_proba[:, 1])

    # Create the ROC curve plot
    plt.figure(figsize=(10, 8))
    plt.plot([0, 1], [0, 1], "k--")
    plt.plot(fpr, tpr, label="AUC = {:.2f}".format(auc))
    plt.xlabel("False Positive Rate", fontsize=14)
    plt.ylabel("True Positive Rate", fontsize=14)
    plt.title("Logistic Regression ROC Curve", fontsize=16)
    plt.legend(loc=4)
    plt.show()

    # Predict the labels for the test set
    y_pred = logreg.predict(test_x)

    # Print evaluation metrics
    print('Recall Score: {:0.3f}'.format(recall_score(test_y, y_pred, average='macro')))
    print('Precision Score: {:0.3f}'.format(precision_score(test_y, y_pred, average='macro')))
    print('F1 Score: {:0.3f}'.format(f1_score(test_y, y_pred, average='macro')))
    print('Accuracy Score: {:0.3f}'.format(accuracy_score(test_y, y_pred)))
    print('AUC score: {:0.3f}'.format(auc))
    print('Training set:', train_x.shape[0])
    print('Test set:', test_x.shape[0])



In [ ]:
# Statistics codes:
# Example



import pandas as pd
import scipy.stats as stats
from pyspark.sql import SparkSession


def unnamed_4(ALL_PATIENTS_SUMMARY):


    """
    Analyze age differences between racial groups from a PySpark DataFrame.
    - Shows age statistics grouped by race.
    - Performs a two-sample t-test on age between two selected races.
    """

    # Convert Spark DataFrame to Pandas
    final_pd = ALL_PATIENTS_SUMMARY.select("age", "race_ethnicity").toPandas()

    # Display unique race values for reference
    print("Available races:", final_pd["race_ethnicity"].unique())

    # Group statistics by race
    grouped = final_pd.groupby("race_ethnicity")["age"].describe()
    print("\nAge summary statistics by race_ethnicity:\n", grouped)

    # Example two-group comparison (adjust race names to fit your data)
    race1 = "White Non-Hispanic"
    race2 = "Hispanic or Latino Any Race"

    age_race1 = final_pd[final_pd["race_ethnicity"] == race1]["age"]
    age_race2 = final_pd[final_pd["race_ethnicity"] == race2]["age"]

    # Drop NaN values just in case
    age_race1 = age_race1.dropna()
    age_race2 = age_race2.dropna()

    # Welch's t-test (assumes unequal variances)
    t_stat, p_val = stats.ttest_ind(age_race1, age_race2, equal_var=False)

    print(f"\nTwo-sample t-test between '{race1}' and '{race2}':")
    print("T-statistic:", t_stat)
    print("P-value:", p_val)


